# This notebook has all of the steps needed to configure your workspace and download imagery for processing and analysis

## import all necessary packages

In [43]:
import os
import numpy as np
from src.landsatUtil.landsat.downloader import Downloader
from src.utils.data_directory_manager import DataDirectoryManager
from src.utils.image_correction import LandsatTOACorrecter
from src.models.antarctic_rock_outcrop_os import OutcropLabeler
import rasterio as rio
import rasterio.mask
import numpy as np
import fiona
from progress.bar import FillingSquaresBar

In [42]:
# !pip install usgs
# !pip install homura
# !pip install geocoder
# !pip install rasterio
# !pip install fiona
# !pip install progress

  Using cached https://files.pythonhosted.org/packages/38/ef/2e887b3d2b248916fc2121889ce68af8a16aaddbe82f9ae6533c24ff0d2b/progress-1.5.tar.gz
  Created wheel for progress: filename=progress-1.5-cp37-none-any.whl size=8075 sha256=f5ff3bb8dd9abd656ca895efe9adc494f943d866d4356c18505ad7dd84404b55
  Stored in directory: /home/sam/.cache/pip/wheels/6c/c8/80/32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
Successfully built progress


## Define functions

In [36]:
def rasterize_label(raster_path, vector_path):
    with fiona.open(vector_path) as vectors:
        shapes = [features['geometry'] for features in vectors]
    
    with rio.open(raster_path) as base:
        meta = base.meta.copy()
        output = (rasterio.mask.mask(base, shapes, crop=True, indexes=1)[0] > 0).astype(rio.uint8)
    
    output = np.expand_dims(output, axis=0)
    return output, meta

## Configure project directory

In [20]:
project_directory = "/home/sam/ant/"
dm = DataDirectoryManager(project_directory)

## Load list of scene ids to be manipulated
### This cell should create a list of scene ids from any source file that can be used to download scenes, correct them, label them, etc

In [21]:
# This particular example loads a list from a file where each line is a single scene id
scene_file_path = os.path.join(dm.project_dir, "qc_ids.txt")
with open(scene_file_path, 'r') as scene_file:
    scene_ids = [i[:-1] for i in scene_file]

In [46]:
scene_ids = ['LC80311222014338LGN00', 'LC82091172014001LGN00', 'LC82131132013362LGN00', 'LC81041072013303LGN00']
bands = [1,2,3,4,5,6,7,9,10,11]

## Download scenes by id
### specific bands can be downloaded by introducing a second parameter
### It is strongly recommended that you specify bands because the provided scenes come as .tifs instead of all bands provided as a .tar.bz file that must be extracted (this extraction takes a very long time)

Landsat band information can be found here: https://www.usgs.gov/land-resources/nli/landsat/landsat-8?qt-science_support_page_related_con=0#qt-science_support_page_related_con

In [54]:
# specify dm.download_dir if not providing a band parameter
downloader = Downloader(download_dir = dm.raw_image_dir) 
downloader.download(scene_ids, bands=bands.copy())

===> Source: AWS S3
===> Downloading: LC80311222014338LGN00_B1.TIF
     LC80311222014338LGN00_B1.TIF already exists on your system
     stored at /home/sam/ant/raw/LC80311222014338LGN00
===> Downloading: LC80311222014338LGN00_B2.TIF
     LC80311222014338LGN00_B2.TIF already exists on your system
     stored at /home/sam/ant/raw/LC80311222014338LGN00
===> Downloading: LC80311222014338LGN00_B3.TIF
     LC80311222014338LGN00_B3.TIF already exists on your system
     stored at /home/sam/ant/raw/LC80311222014338LGN00
===> Downloading: LC80311222014338LGN00_B4.TIF
     LC80311222014338LGN00_B4.TIF already exists on your system
     stored at /home/sam/ant/raw/LC80311222014338LGN00
===> Downloading: LC80311222014338LGN00_B5.TIF
     LC80311222014338LGN00_B5.TIF already exists on your system
     stored at /home/sam/ant/raw/LC80311222014338LGN00
===> Downloading: LC80311222014338LGN00_B6.TIF
     LC80311222014338LGN00_B6.TIF already exists on your system
     stored at /home/sam/ant/raw/LC8031

     LC81041072013303LGN00_B4.TIF already exists on your system
     stored at /home/sam/ant/raw/LC81041072013303LGN00
===> Downloading: LC81041072013303LGN00_B5.TIF
     LC81041072013303LGN00_B5.TIF already exists on your system
     stored at /home/sam/ant/raw/LC81041072013303LGN00
===> Downloading: LC81041072013303LGN00_B6.TIF
     LC81041072013303LGN00_B6.TIF already exists on your system
     stored at /home/sam/ant/raw/LC81041072013303LGN00
===> Downloading: LC81041072013303LGN00_B7.TIF
     LC81041072013303LGN00_B7.TIF already exists on your system
     stored at /home/sam/ant/raw/LC81041072013303LGN00
===> Downloading: LC81041072013303LGN00_B9.TIF
     LC81041072013303LGN00_B9.TIF already exists on your system
     stored at /home/sam/ant/raw/LC81041072013303LGN00
===> Downloading: LC81041072013303LGN00_B10.TIF
     LC81041072013303LGN00_B10.TIF already exists on your system
     stored at /home/sam/ant/raw/LC81041072013303LGN00
===> Downloading: LC81041072013303LGN00_B11.TIF
 

['/home/sam/ant/raw/LC80311222014338LGN00',
 '/home/sam/ant/raw/LC82091172014001LGN00',
 '/home/sam/ant/raw/LC82131132013362LGN00',
 '/home/sam/ant/raw/LC81041072013303LGN00']

# Process downloaded scenes
## only run this cell if downloading .tar.bz files into dm.download_dir

In [ ]:
dm.untar_scenes(scene_ids)

# Correct scenes

In [9]:
for i in scene_ids:
    corrector = LandsatTOACorrecter(i)
    corrector.correct_toa_brightness_tem(dm.corrected_image_dir)
    corrector.correct_toa_reflectance(dm.corrected_image_dir)

TypeError: __init__() missing 1 required positional argument: 'scene_path'

# Rasterize vector labels

# stack bands and split into chunks along with labels

### Define output path

In [26]:
stack_dir = os.path.join(dm.project_dir, "stacked_chunks")

### Configure output directories

In [27]:
for i in scene_ids:
    scene_stack_path = os.path.join(stack_dir, i)
    if not os.path.exists(scene_stack_path):
        os.mkdir(scene_stack_path)

### rasterize label on scene extent

In [37]:
for i in scene_ids:
    reference_scene_path = os.path.join(dm.raw_image_dir, i, i + "_B2.TIF")
    label_path = os.path.join(dm.label_dir, i + "_label.TIF")
    if not os.path.exists(label_path):
        try:
            label, meta = rasterize_label(reference_scene_path, dm.outcrop_shape_path)
            
            meta['dtype'] = label.dtype
            meta['count'] = 1
            
            with rio.open(label_path, 'w', **meta) as dst:
                dst.write(label)
        except ValueError as e:
            with rio.open(band_2_path) as rockless:
                meta = rockless.meta.copy()
                # change shape here to 3d array (width, height, 1). Then dst.write(label) will work for both cases
                zero_shape = (meta["width"], meta["height"])
            label = np.zeros(zero_shape, np.int8)
            
            meta['dtype'] = label.dtype
            meta['count'] = 1
            with rio.open(label_path, 'w', **meta) as dst:
                dst.write(label, 1)

## Sanity check that rasterization cell worked

In [39]:
for i in scene_ids:
    label_path = os.path.join(dm.label_dir, i + "_label.TIF")
    with rio.open(label_path) as label_file:
        label = label_file.read(1)
    print(np.sum(label))

154604
172169
13618
28648


### create band stacks, load labels, break stacks and labels into chunks of (512, 512) pixels

In [66]:
for i in scene_ids:
    label_path = os.path.join(dm.label_dir, i + "_label.TIF")
    with rio.open(label_path) as label_file:
        label = label_file.read(1)
    
    band_rasters = []
    for b in bands:
        band_path = os.path.join(dm.raw_image_dir, i, "{}_B{}.TIF".format(i, b))
        with rio.open(band_path) as band_file:
            band_rasters.append(band_file.read(1))
    
    band_stack = np.stack(band_rasters, axis=0).transpose(1,2,0)
    band_rasters.clear()
    
    scene_height = band_stack.shape[0]
    scene_width = band_stack.shape[1]
    
    chunk_height = 512
    chunk_width = 512
    
    vertical_chunks = scene_height // chunk_height
    horizontal_chunks = scene_width // chunk_width
    
    scene_chunk_dir = os.path.join(stack_dir, i)
    
    with FillingSquaresBar('Processing', max=vertical_chunks * horizontal_chunks) as bar:
        for j in range(vertical_chunks):
            for k in range(horizontal_chunks):
                row_index = j * chunk_height
                col_index = k * chunk_width
                
                band_chunk = band_stack[row_index: row_index + chunk_height, col_index: col_index + chunk_width, :]
                label_chunk = label[row_index: row_index + chunk_height, col_index: col_index + chunk_width]
                
                data_pixels = np.where(band_chunk > 0)
                data_pixel_count = np.sum(data_pixels)
                
                band_chunk_path = os.path.join(scene_chunk_dir, "chunk_{}_{}.npy".format(j, k))
                label_chunk_path = os.path.join(scene_chunk_dir, "chunk_{}_{}_label.npy".format(j, k))
                
                if data_pixel_count > 0:
                    np.save(band_chunk_path, band_chunk, allow_pickle=True)
                    np.save(label_chunk_path, label_chunk, allow_pickle=True)
                bar.next()

## Sanity check that stacking and chunking worked

In [67]:
for i in scene_ids:
    scene_chunk_dir = os.path.join(stack_dir, i)
    print(os.listdir(scene_chunk_dir))

['chunk_10_12.npy', 'chunk_3_10.npy', 'chunk_4_4_label.npy', 'chunk_8_1_label.npy', 'chunk_10_2.npy', 'chunk_8_15.npy', 'chunk_4_5_label.npy', 'chunk_7_6_label.npy', 'chunk_5_0.npy', 'chunk_5_9_label.npy', 'chunk_9_9.npy', 'chunk_3_4.npy', 'chunk_7_0.npy', 'chunk_5_2_label.npy', 'chunk_4_6.npy', 'chunk_7_1_label.npy', 'chunk_5_10.npy', 'chunk_6_7_label.npy', 'chunk_8_11.npy', 'chunk_13_6_label.npy', 'chunk_5_6.npy', 'chunk_0_10.npy', 'chunk_1_10_label.npy', 'chunk_10_5_label.npy', 'chunk_8_5_label.npy', 'chunk_14_7.npy', 'chunk_5_5_label.npy', 'chunk_11_4.npy', 'chunk_8_10_label.npy', 'chunk_4_10.npy', 'chunk_7_8.npy', 'chunk_8_2.npy', 'chunk_6_1_label.npy', 'chunk_4_9_label.npy', 'chunk_7_12_label.npy', 'chunk_12_6_label.npy', 'chunk_7_14_label.npy', 'chunk_6_11.npy', 'chunk_11_5_label.npy', 'chunk_10_8.npy', 'chunk_5_1.npy', 'chunk_13_3_label.npy', 'chunk_9_2.npy', 'chunk_6_12_label.npy', 'chunk_6_10_label.npy', 'chunk_1_8.npy', 'chunk_13_5_label.npy', 'chunk_5_1_label.npy', 'chunk_9